This notebook will contain the ingestion of data from Toronto and Manhatten. I will be using code from the lab to get the Manhatten data and the assignment from week 3 to get the data for Toronto.

The two cities' dataframes will then be joined together and the latitude/longitude removed before clustering.

<br>

<B> Ingesting the Toronto data:

In [1]:
import numpy as np
import pandas as pd
import requests

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'})
column_names = [item.get_text() for item in My_table.find_all('th')]
column_names[2] = column_names[2][:-1]
contents = [item.get_text() for item in My_table.find_all('td')]
values=[]
for item in My_table.select('td'):
    temp = item.get_text()
    values.append(temp)
data = np.reshape(values,(289,3))
df = pd.DataFrame(data = data, columns = column_names)
new_df= df[df.Borough != 'Not assigned'].reset_index()
new_df['Neighborhood'] = new_df['Neighbourhood'].map(lambda x:  str(x)[:-1])
Toronto_df = new_df.groupby(['Postcode','Borough'],as_index = False, sort=False).agg(lambda x: ','.join(x))
Toronto_loc_csv = pd.read_csv("C:Documents/Geospatial_Coordinates.csv")
Toronto_final = pd.merge(Toronto_df, Toronto_loc_csv, left_on = 'Postcode', right_on = 'Postal Code')
Toronto_only = Toronto_final[Toronto_final['Borough'].str.contains("Downtown")]
Toronto_only.head()

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

,Postcode,Borough,Neighborhood,Neighbourhood,Postal Code,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park","Harbourfront\n,Regent Park\n",M5A,43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District","Ryerson\n,Garden District\n",M5B,43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,St. James Town\n,M5C,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,Berczy Park\n,M5E,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,Central Bay Street\n,M5G,43.657952,-79.387383


In [2]:
Toronto_new = Toronto_only[['Borough','Neighborhood','Latitude','Longitude']]
Toronto_new.head()

,Borough,Neighborhood,Latitude,Longitude
2,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,Downtown Toronto,St. James Town,43.651494,-79.375418
20,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [3]:
CLIENT_ID = 'FN5210I2MSNK4CQKXOCUUNBZVVIR2IUINODW4RZDFMJPQARS' # your Foursquare ID
CLIENT_SECRET = '1T3MHIQSPVYZU2JHPILOHFHVA3EY5LAKQXNT5X3NJER2XCHN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [4]:
Toronto_venues = getNearbyVenues(names=Toronto_new['Neighborhood'],
                                   latitudes=Toronto_new['Latitude'],
                                   longitudes=Toronto_new['Longitude']
                                  )


Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


In [5]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


<B> Ingesting the Manhattan data:

In [6]:
import json # library to handle JSON files
with open("C:/Users/markt/documents/nyu-2451-34572-geojson.json","r") as read_file:
    newyork_data = json.load(read_file)

In [7]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [8]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


<B> Combining the cities

In [9]:
the_one_city = pd.concat([Toronto_venues,manhattan_venues])

In [10]:
# one hot encoding
the_one_city_onehot = pd.get_dummies(the_one_city[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
the_one_city_onehot['Neighborhood'] = the_one_city['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [the_one_city_onehot.columns[-1]] + list(the_one_city_onehot.columns[:-1])
the_one_city_onehot = the_one_city_onehot[fixed_columns]

the_one_city_grouped = the_one_city_onehot.groupby('Neighborhood').mean().reset_index()
the_one_city_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Vietnamese Restaurant,Volleyball Court,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.010000
1,Battery Park City,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.030928,0.0,0.010309
2,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000
3,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.0,0.0,0.0,0.0,0.071429,0.071429,0.071429,0.142857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000
4,"Cabbagetown,St. James Town",0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000


In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [12]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = the_one_city_grouped['Neighborhood']

for ind in np.arange(the_one_city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(the_one_city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Burger Joint,Restaurant,Hotel,American Restaurant,Asian Restaurant
1,Battery Park City,Park,Coffee Shop,Hotel,Wine Shop,Italian Restaurant,Gym,Food Truck,Shopping Mall,Burger Joint,Sandwich Place
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Bakery,Italian Restaurant,Steakhouse,Café,Cheese Shop,Farmers Market
3,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Boutique,Harbor / Marina,Sculpture Garden,Airport Food Court,Plane
4,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Park,Pizza Place,Flower Shop,Café,Bakery,Pub,Italian Restaurant,Diner


In [13]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [14]:
# set number of clusters
kclusters = 10

the_one_city_grouped_clustering = the_one_city_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(the_one_city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 5, 8, 5, 0, 5, 0, 2, 2])

In [15]:
the_one_city_data = pd.concat([manhattan_data,Toronto_new])

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [17]:
the_one_city_merged = the_one_city_data

# merge grouped with merged data to add latitude/longitude for each neighborhood
the_one_city_merged = the_one_city_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

the_one_city_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Discount Store,Coffee Shop,Yoga Studio,Clothing Store,Donut Shop,Tennis Stadium,Diner,Sandwich Place,Shoe Store,Department Store
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,American Restaurant,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop,Noodle House,Salon / Barbershop,Hotpot Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Mobile Phone Shop,Bakery,Donut Shop,Latin American Restaurant,Deli / Bodega,Shoe Store,Chinese Restaurant,Tapas Restaurant,Sandwich Place
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Café,Lounge,Deli / Bodega,Pizza Place,Spanish Restaurant,Frozen Yogurt Shop,Wine Bar,Pharmacy,Chinese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Yoga Studio,School,Sandwich Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar


In [29]:
the_one_city_merged.loc[the_one_city_merged['Cluster Labels'] == 0, the_one_city_merged.columns[[0] + [1] + list(range(5, the_one_city_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,Discount Store,Coffee Shop,Yoga Studio,Clothing Store,Donut Shop,Tennis Stadium,Diner,Sandwich Place,Shoe Store,Department Store
2,Manhattan,Washington Heights,Café,Mobile Phone Shop,Bakery,Donut Shop,Latin American Restaurant,Deli / Bodega,Shoe Store,Chinese Restaurant,Tapas Restaurant,Sandwich Place
4,Manhattan,Hamilton Heights,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Yoga Studio,School,Sandwich Place,Caribbean Restaurant,Chinese Restaurant,Cocktail Bar
6,Manhattan,Central Harlem,African Restaurant,Public Art,American Restaurant,Chinese Restaurant,Seafood Restaurant,French Restaurant,Cosmetics Shop,Cafeteria,Park,Beer Bar
15,Manhattan,Midtown,Clothing Store,Hotel,Theater,Steakhouse,Coffee Shop,Spa,Cocktail Bar,Bookstore,Bakery,American Restaurant
20,Manhattan,Lower East Side,Art Gallery,Ramen Restaurant,Café,Chinese Restaurant,Coffee Shop,Shoe Store,Sandwich Place,Pizza Place,Bakery,Japanese Restaurant
22,Manhattan,Little Italy,Bakery,Café,Chinese Restaurant,Seafood Restaurant,Ice Cream Shop,Sandwich Place,Bubble Tea Shop,Salon / Barbershop,Pizza Place,Mediterranean Restaurant
23,Manhattan,Soho,Clothing Store,Boutique,Women's Store,Coffee Shop,Men's Store,Mediterranean Restaurant,Italian Restaurant,Spa,Shoe Store,Art Gallery
25,Manhattan,Manhattan Valley,Coffee Shop,Pizza Place,Deli / Bodega,French Restaurant,Indian Restaurant,Café,Chinese Restaurant,Yoga Studio,Thai Restaurant,Playground
30,Manhattan,Carnegie Hill,Pizza Place,Coffee Shop,Café,Cosmetics Shop,Bar,Yoga Studio,Bookstore,Gym,Spa,French Restaurant


In [30]:
the_one_city_merged.loc[the_one_city_merged['Cluster Labels'] == 1, the_one_city_merged.columns[[0] + [1] + list(range(5, the_one_city_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Manhattan,Stuyvesant Town,Bar,Park,Playground,Heliport,Harbor / Marina,Basketball Court,Cocktail Bar,Coffee Shop,Pet Service,Baseball Field


In [31]:
the_one_city_merged.loc[the_one_city_merged['Cluster Labels'] == 2, the_one_city_merged.columns[[0] + [1] + list(range(5, the_one_city_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Manhattan,Chinatown,Chinese Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,American Restaurant,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop,Noodle House,Salon / Barbershop,Hotpot Restaurant
14,Manhattan,Clinton,Theater,American Restaurant,Gym / Fitness Center,Italian Restaurant,Coffee Shop,Wine Shop,Indie Theater,Gym,Spa,Hotel
16,Manhattan,Murray Hill,Coffee Shop,Hotel,Japanese Restaurant,Bar,French Restaurant,Italian Restaurant,Spa,Gym / Fitness Center,Gym,Sandwich Place
17,Manhattan,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Nightclub,Bakery,American Restaurant,Seafood Restaurant,Hotel,Theater,Market
19,Manhattan,East Village,Bar,Ice Cream Shop,Wine Bar,Mexican Restaurant,Cocktail Bar,Ramen Restaurant,Pizza Place,Italian Restaurant,Coffee Shop,Chinese Restaurant
21,Manhattan,Tribeca,Italian Restaurant,Park,Café,Spa,American Restaurant,Boutique,Wine Bar,Gym,Greek Restaurant,Coffee Shop
27,Manhattan,Gramercy,Italian Restaurant,Thrift / Vintage Store,Cocktail Bar,Bagel Shop,Coffee Shop,American Restaurant,Mexican Restaurant,Wine Shop,Restaurant,Grocery Store
28,Manhattan,Battery Park City,Park,Coffee Shop,Hotel,Wine Shop,Italian Restaurant,Gym,Food Truck,Shopping Mall,Burger Joint,Sandwich Place
29,Manhattan,Financial District,Coffee Shop,Hotel,Bar,Wine Shop,Gym,Steakhouse,Café,Pizza Place,Food Truck,Park
33,Manhattan,Midtown South,Korean Restaurant,Japanese Restaurant,Hotel Bar,Hotel,Boutique,Bakery,Italian Restaurant,Cosmetics Shop,Cocktail Bar,Coffee Shop


In [32]:
the_one_city_merged.loc[the_one_city_merged['Cluster Labels'] == 3, the_one_city_merged.columns[[0] + [1] + list(range(5, the_one_city_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Downtown Toronto,Rosedale,Park,Trail,Playground,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner


In [33]:
the_one_city_merged.loc[the_one_city_merged['Cluster Labels'] == 4, the_one_city_merged.columns[[0] + [1] + list(range(5, the_one_city_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Manhattan,Inwood,Mexican Restaurant,Café,Lounge,Deli / Bodega,Pizza Place,Spanish Restaurant,Frozen Yogurt Shop,Wine Bar,Pharmacy,Chinese Restaurant
7,Manhattan,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Coffee Shop,Thai Restaurant,Steakhouse,Donut Shop,Beer Bar,Dance Studio


In [34]:
the_one_city_merged.loc[the_one_city_merged['Cluster Labels'] == 5, the_one_city_merged.columns[[0] + [1] + list(range(5, the_one_city_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Manhattan,Morningside Heights,Coffee Shop,Bookstore,Park,American Restaurant,Tennis Court,Deli / Bodega,Burger Joint,Food Truck,Café,Sandwich Place
2,Downtown Toronto,"Harbourfront,Regent Park",Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Theater,Gym / Fitness Center,Breakfast Spot,Electronics Store
15,Downtown Toronto,St. James Town,Coffee Shop,Restaurant,Hotel,Café,Park,Gastropub,Breakfast Spot,Clothing Store,Cocktail Bar,Italian Restaurant
20,Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Bakery,Italian Restaurant,Steakhouse,Café,Cheese Shop,Farmers Market
24,Downtown Toronto,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Bubble Tea Shop,Burger Joint,Bar,Chinese Restaurant,Spa,Sandwich Place,Salad Place
36,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",Coffee Shop,Hotel,Aquarium,Pizza Place,Café,Brewery,Italian Restaurant,Restaurant,Bakery,Scenic Lookout
42,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Deli / Bodega,Japanese Restaurant,Bakery,Bar
48,Downtown Toronto,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Seafood Restaurant,Italian Restaurant,Bakery,Deli / Bodega
92,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Café,Restaurant,Cocktail Bar,Beer Bar,Seafood Restaurant,Italian Restaurant,Hotel,Park,Fast Food Restaurant
96,Downtown Toronto,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Park,Pizza Place,Flower Shop,Café,Bakery,Pub,Italian Restaurant,Diner


In [35]:
the_one_city_merged.loc[the_one_city_merged['Cluster Labels'] == 6, the_one_city_merged.columns[[0] + [1] + list(range(5, the_one_city_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Manhattan,Roosevelt Island,Coffee Shop,Sandwich Place,Outdoors & Recreation,School,Greek Restaurant,Park,Deli / Bodega,Kosher Restaurant,Liquor Store,Japanese Restaurant


In [36]:
the_one_city_merged.loc[the_one_city_merged['Cluster Labels'] == 7, the_one_city_merged.columns[[0] + [1] + list(range(5, the_one_city_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattan,Manhattanville,Coffee Shop,Mexican Restaurant,Seafood Restaurant,Italian Restaurant,Food Court,Dumpling Restaurant,Climbing Gym,BBQ Joint,Chinese Restaurant,Laundry Service
8,Manhattan,Upper East Side,Italian Restaurant,Exhibit,Juice Bar,Bakery,Coffee Shop,French Restaurant,Art Gallery,Hotel,Boutique,Gym / Fitness Center
9,Manhattan,Yorkville,Italian Restaurant,Bar,Gym,Coffee Shop,Pizza Place,Mexican Restaurant,Deli / Bodega,Japanese Restaurant,Sushi Restaurant,Park
10,Manhattan,Lenox Hill,Italian Restaurant,Sushi Restaurant,Coffee Shop,Gym / Fitness Center,Pizza Place,Sporting Goods Shop,Burger Joint,Gym,Cocktail Bar,Cosmetics Shop
12,Manhattan,Upper West Side,Italian Restaurant,Bar,Indian Restaurant,Wine Bar,Burger Joint,Coffee Shop,Vegetarian / Vegan Restaurant,Bakery,Sushi Restaurant,Dessert Shop
13,Manhattan,Lincoln Square,Theater,Gym / Fitness Center,Plaza,Italian Restaurant,Concert Hall,Café,French Restaurant,Opera House,Indie Movie Theater,Performing Arts Venue
18,Manhattan,Greenwich Village,Italian Restaurant,French Restaurant,Sushi Restaurant,Clothing Store,Chinese Restaurant,Bakery,Indian Restaurant,Café,American Restaurant,Boutique
24,Manhattan,West Village,Italian Restaurant,New American Restaurant,Gastropub,Wine Bar,Cosmetics Shop,Jazz Club,French Restaurant,Park,Bakery,Speakeasy
31,Manhattan,Noho,Italian Restaurant,French Restaurant,Coffee Shop,Cocktail Bar,Boutique,Hotel,Gift Shop,American Restaurant,Bookstore,Rock Club
32,Manhattan,Civic Center,Gym / Fitness Center,Bakery,Italian Restaurant,Cocktail Bar,French Restaurant,Yoga Studio,Sandwich Place,Park,Spa,Coffee Shop


In [37]:
the_one_city_merged.loc[the_one_city_merged['Cluster Labels'] == 8, the_one_city_merged.columns[[0] + [1] + list(range(5, the_one_city_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Boutique,Harbor / Marina,Sculpture Garden,Airport Food Court,Plane


In [38]:
the_one_city_merged.loc[the_one_city_merged['Cluster Labels'] == 9, the_one_city_merged.columns[[0] + [1] + list(range(5, the_one_city_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,Christie,Grocery Store,Café,Park,Diner,Italian Restaurant,Athletics & Sports,Coffee Shop,Nightclub,Restaurant,Baby Store


<B> Separating Toronto and Manhattan for generating maps

In [41]:
map_data = the_one_city_merged.iloc[:,0:4]
toronto_map_data = map_data[map_data['Borough'].str.contains('Downtown')]
manhattan_map_data = map_data[map_data['Borough'].str.contains('Manhattan')]

<B> Creating Manhattan map

In [48]:
!conda install -c conda-forge folium=0.6.0 --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\markt\Anaconda3

  added / updated specs:
    - folium=0.6.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.6.0               |             py_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          51 KB

The following packages will be UPDATED:

  folium                                         0.5.0-py_0 --> 0.6.0-py_0




folium-0.6.0         | 51 KB     |            |   0% 
folium-0.6.0         | 51 KB     | ##3        |  24% 
folium-0.6.0         | 51 KB     | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [52]:
import folium

# create map
manhatten_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
manhattan_markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_map_data['Latitude'], manhattan_map_data['Longitude'], manhattan_map_data['Neighborhood'], manhattan_map_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
manhatten_map_clusters

NameError: name 'base_events' is not defined

In [ ]:
# create map
toronto_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
toronto_markers_colors = []
for lat, lon, poi, cluster in zip(toronto_map_data['Latitude'], toronto_map_data['Longitude'], toronto_map_data['Neighborhood'], toronto_map_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
toronto_map_clusters